In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:80% !important;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-header{margin-top:150px;}
</style>
<style>
span.toc-item-num{
display:none;
}
</style>
"""))

<b><font size="6" color="red">ch10. 데이터베이스 연동</font></b>

# 1절. SQLite
- SQLite 데이터 베이스는 별도의 DBMS없이 SQL을 이용하여 DB 엑세스 할 수 있도록 만든 간단한 디스크 기반의 DB제공
- C 라이브러리
- SQLite는 프로토 타입을 만들 때 사용
- 프로젝트 단계 : 분석 -> 설계 -> 구현 -> 테스트 -> 고객에게 배포 -> 유지보수
                -  프로토타입         시제품          완제품(Oracle, Mysql, Maria...)
- 
- [DB browser for SQLite](https://sqlitebrowser.org/dl/)

## 1.1 SQLite browser 설치 및 sqlite3 패키지 load

In [4]:
import sqlite3
sqlite3.sqlite_version # sqlite3 패키지의 버전

'3.40.1'

In [ ]:
import pandas as pd
pd.__version__

In [ ]:
import numpy as np
np.__version__

## 1.2 데이터베이스 생성 및 연결
- SQLite DB연결시, DB파일이 있으면 연결, DB파일이 없으면 빈 DB파일 생성

In [10]:
# DB 연결
conn = sqlite3.connect('data/ch10_example.db')
conn

In [11]:
# 커서 객체 생성 커서는 SQL문을 실행시키고, 결과데이터를 조회(select, 그외)하는데 사용하는 객체

cursor = conn.cursor()
cursor


In [ ]:
cursor.execute('DROP TABLE MEMBER')

In [ ]:
cursor.execute('''
    CREATE TABLE MEMBER (
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )

''')

In [ ]:
cursor.execute('INSERT INTO MEMBER VALUES(\'홍길동\',20,\'h@h.com\')')
print('수행 결과 행수 :', cursor.rowcount)
sql = "INSERT INTO MEMBER VALUES('신길동',25, 's@s.com')"
cursor.execute(sql)
print('수행 결과 행수 :', cursor.rowcount)
cursor.execute('INSERT INTO MEMBER VALUES(\'긴길동\',20,\'h@h.com\')')
print('수행 결과 행수 :', cursor.rowcount)

In [ ]:
conn.commit() # conn.rolback() DML문에서만 commit

In [ ]:
# SQL 전송 결과는 cursor가 가리킴
cursor.execute("SELECT * FROM MEMBER ORDER BY NAME")

In [ ]:
# insert, update, delete 문 실행 결과: cursor.rowcount
# select문 실행 결과를 받는 함수들
    ## fetchone() : 결과를 한행씩 받을때 (튜플)
    ## fetchall() : 결과를 모두 받을 때 (튜플 list)
    ## fetchmany(n) : 결과를 n행 받을때 (튜플 list)
print(cursor.fetchall())

In [ ]:
print(cursor.fetchall()) # 한번 소요된 cursor 객체는 다시 fetch할 수 없음

In [ ]:
cursor.execute("SELECT NAME,AGE,EMAIL FROM MEMBER ORDER BY AGE")
members = cursor.fetchall()
members # 튜플리스트

In [ ]:
# 한줄씩 읽기
cursor.execute("SELECT NAME, AGE, EMAIL FROM MEMBER ORDER BY AGE")
member_list = []
while True:
    member = cursor.fetchone() # SQL문 수행 결과 한줄 가져오기
    if member is None:
        break
    member_list.append({'name':member[0],'age':member[1],'email':member[2]}

In [ ]:
member_list

In [ ]:
# 최상위 n행 읽기
cursor.execute("SELECT NAME, AGE,EMAIL FROM MEMbER ORDER BY AGE")
for member in cursor.fetchmany(10):
    print(member)

In [6]:
class Member:
    'Member 테이블의 내용을 받을 객체 타임'
    def __init__(self,name,age,email):
        self.name=name
        self.age=age
        self.email=email
    def as_dic(self): # insert문 전송시 필요
        return{'name':self.name,
                'age': self.age,
               'email':self.email}
    def __str__(self):
        return "{}\t{}\t{}".format(self.name,self.age,self.email)
    
def to_member(*row): # 튜플 데이터를 매개변수로 받아 Member형 객체로 return
    return Member(row[0],row[1],row[2])

In [ ]:
dbreadmember = ('홍길동', 20,'h@h.com')
m = to_member(*dbreadmember) # 튜플 언패킹
print(m)
print(m.as_dic())

In [ ]:
# DB검색 결과를 객체 list 로
cursor.execute('SELECT NAME, AGE, EMAIL FROM MEMBER')
member_list = [] # sql 문 수행 결과를 담을 객체 list
members = cursor.fetchall() #튜플 list
#print(members)
for member in members:
    member_list.append(to_member(*member))

In [ ]:
type(members[0]), type(member_list[0])

In [ ]:
for member in member_list:
    print(member)

In [ ]:
cursor.close() # cursor.close()는 생략 가능
conn.close()

## 1.3 SQL구문에 파라미터 사용하기
- qmark(db에 따라 불가한 경우가 있음)
- named(추천)


In [8]:
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()
cursor.execute('SELECT * FROM MEMBER WHERE NAME IN (\'홍길동\',\'신길동\')')
cursor.fetchall()

[('홍길동', 20, 'h@h.com'),
 ('신길동', 25, 's@s.com'),
 ('홍길동', 20, 'h@h.com'),
 ('신길동', 25, 's@s.com')]

In [ ]:
name1 = input('검색할 이름1:')
name2 = input('검색할 이름2:')
names = (name1,name2)
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN(?,?)",(names))
cursor.fetchall()

In [16]:
# 파라미터 사용하기 : named 방법이용
name = input('검색할 이름은 >')
cursor.execute("SELECT *FROM MEMBER WHERE NAME=:name",{'name':name})
members = cursor.fetchall()
if len(members):
    print(members)
else : 
    print('해당 이름의 데이터가 없습니다')

KeyboardInterrupt: Interrupted by user

In [ ]:
# named 방법
name1 = input('검색할 이름1:')
name2 = input('검색할 이름2:')
names = (name1,name2)
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN(:name1,:name2)",{'name1':name1,
                                                                   'name2':name2})
cursor.fetchall()

In [ ]:
# named 방법
name1 = input('검색할 이름1:')
name2 = input('검색할 이름2:')
names = {'name1':name1,'name2':name2}
#cursor.execute("SELECT * FROM MEMBER WHERE NAME IN(:name1,:name2)",{'name1':name1,
#                                                                   'name2':name2})
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN(:name1,:name2)",names)
cursor.fetchall()

In [12]:
# MEMBER 테이블에 입력(사용자로부터 이름 나이 메일을 입력받아 insert)
try:
    name = input('입력할 이름은:')
    age = int(input('입력할 나이는:'))
except ValueError:
    print('유효하지 않은 나이를 입력하신 경우 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은 :')
inputdata={'name':name,'age':age,'email':email}
inputdata2=(name,age,email) # qmark방식
#cursor.execute('INSERT INTO MEMBER VALUES(?,?,?)', inputdata2)
cursor.execute('INSERT INTO MEMBER VALUES(:name,:age,:email)', 
               inputdata)
conn.commit()
if cursor.rowcount ==1:
    print('저장완료')


입력할 이름은:김수민
입력할 나이는:ㅁ
유효하지 않은 나이를 입력하신 경우 18세로 초기화
입력할 메일은 :s@s.com
저장완료


In [77]:
cursor.execute('SELECT * FROM MEMBER')
cursor.fetchall()

[('최수민', '010-4413-3419', 'soom@siim.com', 27, 1, 'soom')]

In [18]:
# MEMBER 테이블에 입력(사용자로부터 이름 나이 메일을 입력받아 insert)
try:
    name = input('입력할 이름은:')
    age = input('입력할 나이는:')
except ValueError:
    print('유효하지 않은 나이를 입력하신 경우 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은 :')
    
newMember = Member(name,age,email)
print(newMember.as_dic())
cursor.execute('INSERT INTO MEMBER VALUES (:name,:age,:email)',newMember.as_dic())
conn.commit()
if cursor.rowcount==1:
    print('입력성공')

입력할 이름은:ririri
입력할 나이는:23
입력할 메일은 :ri@ri.com
{'name': 'ririri', 'age': '23', 'email': 'ri@ri.com'}
입력성공


In [14]:
# member 테이블에 데이터를 n번 입력하고
# 이름에 0을 입력할때까지 이름, 나이, 메일을 받아 insert
# 이름에 0을 입력하면 이때까지 입력한 데이터들을 출력하고 몇명을 입력했는지도 출력

member_list = [] # 입력한 member들이 저장 될 list변수
while True:
    try:
        name = input('입력할 이름은(종료는 0):')
        if name=='0':
            break
        age = int(input('입력할 나이는'))
    except ValueError:
        print('유효하지 않은 나이입니다. 18세로 초기화')
        age = 18
        
    finally:
        email = input('입력할 메일은 :')
    newMember = {'name':name,'age':age,'email':email}
    member_list.append(newMember)
    cursor.execute('INSERT INTO MEMBER VALUES(:name, :age,:email)',newMember)
conn.commit()
print('입력된 데이터는 ', len(member_list), '명입니다', sep='')
for member in member_list:
    print("{}\t{}\t{}".format(member['name'],member['age'],member['email']))

입력할 이름은(종료는 0):0
입력할 메일은 :a
입력된 데이터는 0명입니다


In [19]:
member_list = [] # 입력한 member들이 저장 될 list변수(딕셔너리 리스트)
while True:
    try:
        name = input('입력할 이름은(종료는 0):')
        if name=='0':
            break
        age = int(input('입력할 나이는'))
        email = input('입력할 메일은 :')
    except ValueError:
        print('유효하지 않은 나이입니다. 18세로 초기화')
        age = 18
        
    finally:  
        newMember = Member(name,age,email)
        member_list.append(newMember)
        cursor.execute('INSERT INTO MEMBER VALUES(:name, :age,:email)',newMember.as_dic())
        print(newMember)
conn.commit()
print('입력된 자료는 다음과 같이 {}명입니다'.format(len(member_list)))
print('이름\t나이\t메일')
for member in member_list:
    print(member)

입력할 이름은(종료는 0):0
0	25	ramaba@ra.com
입력된 자료는 다음과 같이 1명입니다
이름	나이	메일
0	25	ramaba@ra.com


In [20]:
cursor.close()
conn.close()

# 2절. 오라클 데이터 베이스 연결
- [cx_Oracle](https://cx-oracle.readthedocs.io/en/latest/)

In [21]:
import cx_Oracle

In [25]:
# conn 얻어오는 방법 1
oracle_dsn = cx_Oracle.makedsn(host="localhost",port=1521,sid='xe')
print(oracle_dsn)
conn = cx_Oracle.connect("scott","tiger",dsn= oracle_dsn)
conn
# 여기서 예외가 날 경우 VS_redist.x64.exe

(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))


<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [26]:
conn.close()

In [28]:
# conn 얻어오는 방법 2
conn = cx_Oracle.connect("scott","tiger","localhost:1521/xe")
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [29]:
# cursor 객체 생성
cursor= conn.cursor()
sql = "SELECT * FROM EMP"
cursor.execute(sql)
emp = cursor.fetchall()


In [30]:
for e in emp : # 튜플 리스트라 field 가 많은 경우 가독성 다소 떨어짐
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [33]:
import pandas as pd
emp_df = pd.DataFrame(emp)
emp_df.head()

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [34]:
# select문을 수행한 각 필드 정보
cursor.description


[('EMPNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 0),
 ('ENAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 10, 10, None, None, 1),
 ('JOB', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 9, 9, None, None, 1),
 ('MGR', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 1),
 ('HIREDATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1),
 ('SAL', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('COMM', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('DEPTNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 3, None, 2, 0, 1)]

In [36]:
result =[]
for description in cursor.description:
    result.append(description[0])
result

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [38]:
[row[0] for row in cursor.description]

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [42]:
emp_df.columns

RangeIndex(start=0, stop=8, step=1)

In [44]:
emp_df.columns= [row[0] for row in cursor.description]
emp_df.sample()# 임의의 한줄 출력

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
12,7902,FORD,ANALYST,7566.0,1981-12-03,3000.0,NaN,20


In [47]:
# 검색할 이름을 사용자 에게 받아 해당 내용을 출력
ename = input('검색할 이름은?')
sql = "SELECT *FROM EMP WHERE ENAME = UPPER(:name)"
cursor.execute(sql,{'name':ename})
emp = cursor.fetchall()
if len(emp):
    for e in emp:
        print(e)
else:
    print('입력하신 이름의 데이터는 없습니다')

검색할 이름은?scott
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)


In [51]:
fieldnames=[description[0] for description in cursor.description]
print(fieldnames)
print(e)

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)


In [52]:
for idx in range(len(e)):
    print("{}:{}".format(fieldnames[idx],e[idx] if e[idx] is not None else ''))

EMPNO:7788
ENAME:SCOTT
JOB:ANALYST
MGR:7566
HIREDATE:1982-12-09 00:00:00
SAL:3000.0
COMM:
DEPTNO:20


In [53]:
for fieldname, data in zip(fieldnames,e):
    print("{}:{}".format(fieldname,data if data is not None else '-'))

EMPNO:7788
ENAME:SCOTT
JOB:ANALYST
MGR:7566
HIREDATE:1982-12-09 00:00:00
SAL:3000.0
COMM:-
DEPTNO:20


In [54]:
cursor.close()
conn.close()

# 3절. 연습문제

In [82]:
class Member:
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = self
    def __str__(self):
        return "{}\t{}\t{}\t{}\t{}\t{}".format(self.name,self.phone,self.email,self.age,self.grade,self.etc)
    def as_dict(self):
        return {'name':self.name,
                'phone':self.phone,
                'email':self.email,
                'age':self.age,
                'grade':self.grade,
               'etc': self.etc}
    def to_member(*row): # 튜플 데이터를 매개변수로 받아 Member형 객체로 반환
        return Member(row[0],row[1],row[2],row[3],row[4],row[5])

In [83]:
# 1.입력
def insert_member():
    name = input('입력할 이름은 :')
    phone = input('입력할 전화번호는 :')
    email = input('입력할 이메일은 :')
    try:
        age = int(input('입력할 나이는 :'))
        if age<0:
            age=0
    except ValueError:
        print('유효한 나이가 아닙니다')
        age = 0
    try:
        grade = int(input('입력할 등급은 :'))
        if grade < 1:
            grade = 1
        if grade >5:
            grade = 5
    except ValueError:
        print('유효한 등급이 아닙니다')
        grade = 1
    etc = input('입력할 기타내용은 :')
    inputm = (name, phone,email,age,grade,etc)
    cursor.execute('INSERT INTO MEMBER VALUES(?,?,?,?,?,?)',inputm)
    conn.commit()
    if cursor.rowcount ==1:
        print('정상적으로 INSERT')
    
    

In [95]:
# 2. 전체 출력
def select_all():
    cursor.execute('SELECT *FROM MEMBER')
    members = cursor.fetchall()
    for member in members:
        print(member)

In [96]:
cursor = conn.cursor()
select_all()

('최수민', '010-4413-3419', 'soom@siim.com', 27, 1, 'soom')


In [104]:
# 3. 이름 검색
def select_name():
    name = input('검색할 이름은?')
    cursor.execute('SELECT * FROM MEMBER WHERE NAME=:name',{'name':name})
    members = cursor.fetchall()
    for member in members:
        print(member)

In [ ]:
# 4. 메일 삭제
def delete_email():
    

In [ ]:
# 5. CSV 내보내기
def save_csv():
    

In [ ]:
# 0. 종료
def close_sql():



In [80]:
def main():
    while True:
        print("1.입력", "2.전체 조회", "3.이름 찾기", "4.메일 삭제", "5.CSV 내보내기", "0.종료",
        sep=" | ")
        try:
            menu = int(input("메뉴 선택: "))
        except:
            print("유효하지 않은 값을 입력하였습니다. 다시 선택해주세요.")
        if menu==1:
            insert_member()
        elif menu==2:
            select_all()
        elif menu==3:
            select_name()
        elif menu==4:
            delete_email()
        elif menu==5:
            save_csv()
        elif menu==0:
            close_sql()
            break



In [105]:
if __name__=='__main__':
    import sqlite3
    global conn
    conn = sqlite3.connect('data/ch10_data.db')
    cursor = conn.cursor()
    main()


1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 3
검색할 이름은?최수민
('최수민', '010-4413-3419', 'soom@siim.com', 27, 1, 'soom')
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 3
검색할 이름은?최수믠
('최수믠', '010-2222-4453', 'min@min.com', 22, 5, 'mmin')
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
유효하지 않은 값을 입력하였습니다. 다시 선택해주세요.


KeyboardInterrupt: Interrupted by user